# DS-SF-34 | 11 | Logistic Regression | Assignment | Starter Code

In [92]:
import os

import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 10)
pd.set_option('display.notebook_repr_html', True)
pd.set_option('display.max_columns', 10)

from sklearn import linear_model, cross_validation

import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')

## Probability, Odds, and Odds Ratios

**Probability:** The number of ways that an event can occur divided by the total number of possible outcomes.

The probability of drawing a red card from a standard deck of cards is 26/52 (50 percent).
The probability of drawing a club from that deck is 13/52 (25 percent).

> ### Question 1.  What's the probability of getting heads in a fair coin flip?

In [93]:
# TODO
def odds(p):
    return p/(1.-p)

def log_odds(p):
    odds_value = odds(p)
    return np.log(odds_value)

def sigmoid(log_odds):
    return 1. / (1+np.exp(-log_odds))

In [94]:
sigmoid(log_odds(.32))

0.32

The odds for an event is the ratio of the number of ways the event can occur compared to the number of ways it does not occur.

For example, using the same events as above, the odds for:
- Drawing a red card from a standard deck of cards is 1:1; and
- Drawing a club from that deck is 1:3.

> ### Question 2.  What's the odds of a fair coin flip?

In [95]:
# TODO
#1:1

> ### Question 3.  Suppose that 18 out of 20 patients in an experiment lost weight while using diet A, while 16 out of 20  lost weight using diet B.  What's the probability of weight loss with diet A?  What's the odds?

In [96]:
# TODO
90%

SyntaxError: invalid syntax (<ipython-input-96-60296dafa3cb>, line 2)

> ### Question 4.  What's the probablity of weight loss with diet B?  What are the odds?

In [ ]:
# TODO
80%

> ### Question 5.  What's the odds ratio?

In [ ]:
# TODO
4:1

## Bank Marketing

In [ ]:
df = pd.read_csv(os.path.join('..', 'datasets', 'dataset-11-bank-marketing.csv'))

In [ ]:
df.info()

> The dataset is related to the direct marketing campaigns (by phone) of a Portuguese banking institution.  The classification goal is to predict if the client will subscribe a term deposit (variable y).

Attributes Information:

- Input variables:
  - [Bank client data]
    - `age` (numeric)
    - `job`: type of job (categorical)
    - `marital`: marital status (categorical)
      - Note: `divorced` means divorced or widowed)
    - `education` (categorical)
    - `default`: has credit in default? (categorical)
    - `balance`: bank account balance (\$)
    - `housing`: has housing loan? (categorical)
    - `loan`: has personal loan? (categorical)
  - [Data related with the last contact of the current campaign]
    - `contact`: contact communication type (categorical)
    - `month`: last contact month of year (categorical)
    - `day_of_week`: last contact day of the week (categorical)
    - `duration`: last contact duration, in seconds (numeric)
      - Important note: this attribute highly affects the output target (e.g., if `duration = 0` then `y = 'no'`). Yet, the duration is not known before a call is performed.  Also, after the end of the call y is obviously known.  Thus, this input should only be included for benchmark purposes and should be discarded if the intention is to have a realistic predictive model.
  - [Other attributes]
    - `campaign`: number of contacts performed during this campaign and for this client (numeric)
    - `pdays`: number of days that passed by after the client was last contacted from a previous campaign (numeric)
      - 999 means client was not previously contacted
    - `previous`: number of contacts performed before this campaign and for this client (numeric)
    - `poutcome`: outcome of the previous marketing campaign (categorical)

- Output variable (desired target):
  - `y`: has the client subscribed a term deposit? (binary)

> Our goal is to develop a model the best predicts the outcome `y`, the success of the marketing campaign.

> ## Question 6.  Remove the categorical variables with the most number of distinct values

In [ ]:
df.drop(columns = ['job','balance','day','month','duration','campaign','pdays','previous'], inplace=True)

In [ ]:
# TODO the below selects all categorical variables
for column in df.select_dtypes(include='object').columns:
    print column, df[column].nunique()
    
yes_no_columns = [
    'default',
    'housing',
    'loan',
    'c'
]



In [ ]:
def convert_to_binary(yesno_string):
    if yesno_string == 'yes':
        return 1
    else:
        return 0

In [ ]:
for yesno_col in yes_no_columns:
    df[yesno_col] = df[yesno_col].map(convert_to_binary)

In [ ]:
df[yes_no_columns]

Answer: TODO

> ## Question 7.  Recode all `yes`/`no` categorical variables with `0` as the most frequent value (then also append `"_no"` to the variable name), and `1` for the other (then leave the name unchanged)

In [ ]:
#See above

> ## Question 8.  Create binary/dummy variables for the other categorical variables

In [ ]:
# TODO
# smart enough to just put in df, and it leaves the numbers alone. only converts categories
dummies = pd.get_dummies(df)

In [85]:
# all the fields are ints now so ready to go!
dummies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4521 entries, 0 to 4520
Data columns (total 19 columns):
age                    4521 non-null int64
default                4521 non-null int64
housing                4521 non-null int64
loan                   4521 non-null int64
c                      4521 non-null int64
marital_divorced       4521 non-null uint8
marital_married        4521 non-null uint8
marital_single         4521 non-null uint8
education_primary      4521 non-null uint8
education_secondary    4521 non-null uint8
education_tertiary     4521 non-null uint8
education_unknown      4521 non-null uint8
contact_cellular       4521 non-null uint8
contact_telephone      4521 non-null uint8
contact_unknown        4521 non-null uint8
poutcome_failure       4521 non-null uint8
poutcome_other         4521 non-null uint8
poutcome_success       4521 non-null uint8
poutcome_unknown       4521 non-null uint8
dtypes: int64(5), uint8(14)
memory usage: 238.5 KB


> ## Question 9.  What should be your baseline for these binary variables (namely, which binary variables should you not include in your model)?

In [ ]:
# TODO
probably c? is that the y variable if the customer signed up?

Answer: TODO

> ## Question 10.  What input variable in the dataset seems to predict the outcome quite well.  Why?

In [65]:
dummies.corr()

,age,default,housing,loan,c,...,contact_unknown,poutcome_failure,poutcome_other,poutcome_success,poutcome_unknown
age,1.000000,-0.017885,-0.193888,-0.011250,0.045092,...,-0.035125,0.012693,-0.026179,0.048626,-0.017414
default,-0.017885,1.000000,0.006881,0.063994,0.001303,...,0.014151,-0.040055,0.005801,-0.022410,0.038997
housing,-0.193888,0.006881,1.000000,0.018451,-0.104683,...,0.215344,0.094246,0.046986,-0.088504,-0.062805
loan,-0.011250,0.063994,0.018451,1.000000,-0.070517,...,-0.005894,-0.011652,-0.018397,-0.050643,0.041107
c,0.045092,0.001303,-0.104683,-0.070517,1.000000,...,-0.139399,0.014556,0.051908,0.283481,-0.162038
...,...,...,...,...,...,...,...,...,...,...,...
contact_unknown,-0.035125,0.014151,0.215344,-0.005894,-0.139399,...,1.000000,-0.218116,-0.125456,-0.104451,0.288110
poutcome_failure,0.012693,-0.040055,0.094246,-0.011652,0.014556,...,-0.218116,1.000000,-0.074419,-0.059752,-0.742918
poutcome_other,-0.026179,0.005801,0.046986,-0.018397,0.051908,...,-0.125456,-0.074419,1.000000,-0.036581,-0.454820
poutcome_success,0.048626,-0.022410,-0.088504,-0.050643,0.283481,...,-0.104451,-0.059752,-0.036581,1.000000,-0.365185


In [64]:
# TODO
poutcome predicts fairly well probably because its the outcome of the last time.

TypeError: 'module' object is not callable

Answer: TODO

> ## Question 11.  Split the dataset into a training set (60%) and a testing set (the rest)

In [ ]:
# TODO


> ## Question 12.  Run a logistic regression with `age`, `marital` (the dummies), `default`, `balance`, `housing`, `loan`, `campaign`, `pdays`, `previous`?

In [ ]:
# TODO

> ## Question 13.  What is your training error?  What is your generalization error?  Does it make sense?

In [ ]:
# TODO

Answer: TODO

> ## Question 14.  Interpret your coefficients. (At least `marital_single`, `campaign`, and `default`).  Does your interpretation  make sense?

In [ ]:
# TODO

Answer: TODO

> ## Question 15.  What is your prediction for a 30 years old single female, a homeowner with a \$1,000 balance in the bank, without a loan, who has never been contacted before, and who has never defaulted

In [ ]:
# TODO

Answer: TODO